In [1]:
!python -m spacy download en_core_web_sm


c:\Users\DELL\AppData\Local\Programs\Python\Python39\python.exe: No module named spacy


In [2]:
import os
import random
##Escoge 100 archivos al azar
folder_path = "reviews"  
all_files = os.listdir(folder_path)
random_files = random.sample(all_files, 100)
print(random_files)

FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: 'reviews'

In [ ]:
##Creacion de diccionario de peliculas
movies_dict = {}
for file in random_files:
    new_file = file.split('_')
    new_new_file = new_file[1].split('.')
    key = new_new_file[0]
    if key in movies_dict:
        movies_dict[key].append(file)
    else:
        movies_dict[key] = [file]
print(movies_dict)


{'1': ['10820_1.txt', '3765_1.txt', '5885_1.txt', '6881_1.txt', '4704_1.txt', '4852_1.txt', '9325_1.txt', '332_1.txt', '11718_1.txt', '5945_1.txt', '3533_1.txt', '6992_1.txt', '2353_1.txt', '2094_1.txt', '12297_1.txt', '2769_1.txt', '1692_1.txt', '3731_1.txt', '6239_1.txt', '5903_1.txt', '1506_1.txt', '4478_1.txt', '3314_1.txt', '4075_1.txt', '7039_1.txt', '3207_1.txt', '4796_1.txt', '308_1.txt', '8013_1.txt', '9944_1.txt', '6641_1.txt', '5864_1.txt', '10784_1.txt'], '4': ['6183_4.txt', '6336_4.txt', '287_4.txt', '6727_4.txt', '11765_4.txt', '12023_4.txt', '1637_4.txt', '5391_4.txt', '3823_4.txt', '2255_4.txt', '8888_4.txt', '10258_4.txt', '4012_4.txt', '2970_4.txt', '649_4.txt', '8194_4.txt', '5694_4.txt', '9545_4.txt', '390_4.txt', '7408_4.txt', '10436_4.txt', '8568_4.txt', '4789_4.txt', '11749_4.txt', '8044_4.txt', '5393_4.txt', '1832_4.txt', '5600_4.txt', '9598_4.txt', '9731_4.txt', '1856_4.txt', '2818_4.txt'], '2': ['11047_2.txt', '4059_2.txt', '2661_2.txt', '8932_2.txt', '11656_2

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')




[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\charl\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\charl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\charl\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import os
import string

# Initialize NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Create a lemmatizer
lemmatizer = WordNetLemmatizer()

# Define the directory containing your movie files
directory = 'reviews/'

# Iterate over movies
for movie in movies_dict:
    moviechosen = movies_dict[movie]
    
    # Create a directory for lemmatized files
    lemmatized_directory = 'lemmatized_reviews/'
    os.makedirs(lemmatized_directory, exist_ok=True)
    
    for file in moviechosen:
        path = os.path.join(directory, file)
        
        with open(path, 'r') as archivo:
            text = archivo.read()
            text=text.lower()
            text=text.replace("< br / >","")
            text=text.replace("<br />","")
            text = text.translate(str.maketrans('', '', string.punctuation))
            tokens = nltk.word_tokenize(text)
            stop_words = set(stopwords.words('english'))
            filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
            lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]
            lemmatized_text = ' '.join(lemmatized_tokens)
            
            # Create a new file with lemmatized content
            lemmatized_path = os.path.join(lemmatized_directory, file)
            with open(lemmatized_path, "w") as output_file:
                output_file.write(lemmatized_text)

print("Lemmatized files have been created in the 'lemmatized_reviews' directory.")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\charl\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\charl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\charl\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Lemmatized files have been created in the 'lemmatized_reviews' directory.


In [ ]:
import os

input_folder = 'lemmatized_reviews/'
output_folder = 'reviews_by_movie/'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

file_contents = {}
for filename in os.listdir(input_folder):
    file_number = filename.split("_")[1].split('.')[0]

    with open(os.path.join(input_folder, filename), 'r') as input_file:
        file_content = input_file.read()

    if file_number not in file_contents:
        file_contents[file_number] = []
    file_contents[file_number].append(file_content)

for file_number, contents in file_contents.items():
    output_file_path = os.path.join(output_folder, f"{file_number}.txt")
    with open(output_file_path, 'w') as output_file:
        output_file.write("\n".join(contents))

print("Files have been grouped and written to separate output files.")

In [ ]:
from collections import Counter
import string

def get_term_frequency_in_file(source_file_path):
    wordcount = {}
    with open(source_file_path) as f:
        translator = str.maketrans('', '', string.punctuation)  # Create a translation table
        for line in f:
            line = line.lower().translate(translator)  # Use the translation table to remove punctuation
            this_wordcount = Counter(line.split())
            wordcount = add_merge_two_dict(wordcount, this_wordcount)
    return wordcount

def add_merge_two_dict(x, y):
    return { k: x.get(k, 0) + y.get(k, 0) for k in set(x) | set(y) }

directory = 'reviews_by_movie/'

for movie in movies_dict:
    get_term_frequency_in_file(movie)
